# get val dataset from train data

In [10]:
# coding:utf-8
import glob
import os.path
import numpy as np
import os
import re
'''
创建验证集bin的pairs.txt
'''
import random
# 图片数据文件夹
INPUT_DATA = r'raw_data/train_data/mtcnn_val/'

rootdir_list = os.listdir(INPUT_DATA)
idsdir_list = []
for region_name in rootdir_list:
    if os.path.isdir(os.path.join(INPUT_DATA, region_name)):
        class_dir_list = os.listdir(os.path.join(INPUT_DATA, region_name))
        for class_name in class_dir_list:
            if os.path.isdir(os.path.join(INPUT_DATA, region_name , class_name)):
                idsdir_list.append(os.path.join(region_name , class_name))
len(idsdir_list)

#raw 16520
#now 14870

1650

In [3]:
count_class = {'Indian':0,
                'Caucasian':0,
                'Asian':0,
                'African':0,
              }
for dir_class in idsdir_list:
    if dir_class.split('/')[0] == 'Indian':
        count_class['Indian'] += 1
    elif dir_class.split('/')[0] == 'Caucasian':
        count_class['Caucasian'] += 1
    elif dir_class.split('/')[0] == 'Asian':
        count_class['Asian'] += 1
    elif dir_class.split('/')[0] == 'African':
        count_class['African'] += 1
count_class

{'Indian': 1923, 'Caucasian': 11326, 'Asian': 1728, 'African': 1543}

In [8]:
#train data : val data = 9:1
import shutil
OUTPUT_DATA = r'raw_data/train_data/mtcnn_val/'
for class_name , class_num in count_class.items():
    # Move src to dst (mv src dst)
    temp_num = int(class_num / 10)
    clss_list = [x for x in idsdir_list if x.split('/')[0] == class_name ]
    temp_dir_list = random.sample(clss_list , temp_num)
    for temp_dir in temp_dir_list:
        shutil.move(INPUT_DATA + temp_dir, OUTPUT_DATA + temp_dir)

# get pair.txt

In [18]:
# coding:utf-8
import glob
import os.path
import numpy as np
import os
import re
'''
创建验证集bin的pairs.txt
'''
import random
# 图片数据文件夹
INPUT_DATA = r'raw_data/train_data/mtcnn_val/'
pairs_file_path = 'ccf_data/pairs_val.txt'

rootdir_list = os.listdir(INPUT_DATA)
idsdir_list = []
for region_name in rootdir_list:
    if os.path.isdir(os.path.join(INPUT_DATA, region_name)):
        class_dir_list = os.listdir(os.path.join(INPUT_DATA, region_name))
        for class_name in class_dir_list:
            if os.path.isdir(os.path.join(INPUT_DATA, region_name , class_name)):
                idsdir_list.append(os.path.join(region_name , class_name))
len(idsdir_list)

1650

In [19]:
count_class = {'Indian':0,
                'Caucasian':0,
                'Asian':0,
                'African':0,
              }
for dir_class in idsdir_list:
    if dir_class.split('/')[0] == 'Indian':
        count_class['Indian'] += 1
    elif dir_class.split('/')[0] == 'Caucasian':
        count_class['Caucasian'] += 1
    elif dir_class.split('/')[0] == 'Asian':
        count_class['Asian'] += 1
    elif dir_class.split('/')[0] == 'African':
        count_class['African'] += 1
count_class

{'Indian': 192, 'Caucasian': 1132, 'Asian': 172, 'African': 154}

In [20]:

id_nums = 12000

def produce_same_pairs():
    matched_result = []  # 同类的匹配对
    for j in range(id_nums):
        id_dir_class = random.choice(idsdir_list)

        id_dir = os.path.join(INPUT_DATA, id_dir_class)

        id_imgs_list = os.listdir(id_dir)

        id_list_len = len(id_imgs_list)
        
        while True:
            id1_img_file = id_imgs_list[random.randint(0,id_list_len-1)]
            id2_img_file = id_imgs_list[random.randint(0,id_list_len-1)]
            if id1_img_file != id2_img_file:
                break
                
        id1_path = os.path.join(id_dir, id1_img_file)
        id2_path = os.path.join(id_dir, id2_img_file)

        same = 1
        #print([id1_path + '\t' + id2_path + '\t',same])
        if (id1_path + '\t' + id2_path + '\t',same) not in matched_result:
            matched_result.append((id1_path + '\t' + id2_path + '\t',same))
        else:
            j -= 1
    return matched_result


def produce_unsame_pairs():
    unmatched_result = []  # 不同类的匹配对
    for j in range(id_nums):
        id1_dir_class = random.choice(idsdir_list)
        id2_dir_class = random.choice(idsdir_list)
        while id1_dir_class == id2_dir_class:
            id1_dir_class = random.choice(idsdir_list)
            id2_dir_class = random.choice(idsdir_list)

        id1_dir = os.path.join(INPUT_DATA, id1_dir_class)
        id2_dir = os.path.join(INPUT_DATA, id2_dir_class)

        id1_imgs_list = os.listdir(id1_dir)
        id2_imgs_list = os.listdir(id2_dir)
        id1_list_len = len(id1_imgs_list)
        id2_list_len = len(id2_imgs_list)

        id1_img_file = id1_imgs_list[random.randint(0, id1_list_len-1)]
        id2_img_file = id2_imgs_list[random.randint(0, id2_list_len-1)]

        id1_path = os.path.join(id1_dir, id1_img_file)
        id2_path = os.path.join(id2_dir, id2_img_file)

        same = 0
        if (id1_path + '\t' + id2_path + '\t',same) not in unmatched_result:
            unmatched_result.append((id1_path + '\t' + id2_path + '\t',same))
        else:
            j -= 1
    return unmatched_result


same_result = produce_same_pairs()
unsame_result = produce_unsame_pairs()

all_result = same_result + unsame_result

random.shuffle(all_result)
#print(all_result)

file = open(pairs_file_path, 'w')
for line in all_result:
    file.write(line[0] + str(line[1]) + '\n')

file.close()

# make train bin

In [21]:
import mxnet as mx
from mxnet import ndarray as nd
import argparse
import pickle
import sys
import os
import numpy as np

INPUT_DATA = r''
finetune_dir = r'ccf_data/'
# parser = argparse.ArgumentParser(description='Package LFW images')
# # general
# parser.add_argument('--data-dir', default=INPUT_DATA, help='')
# parser.add_argument('--image-size', type=str, default='112,96', help='')
# parser.add_argument('--output', default=r'finetune_data/train.bin', help='path to save.')
# args = parser.parse_args()

image_size = '112,112'
output = r'ccf_data/ccf_val.bin'
lfw_dir = INPUT_DATA
image_size = [int(x) for x in image_size.split(',')]


def read_pairs(pairs_filename):
    pairs = []
    with open(pairs_filename, 'r') as f:
        for line in f.readlines():
            pair = line.strip().split()
            pairs.append(pair)
    return np.array(pairs)

def get_paths(data_dir, pairs, file_ext):
    nrof_skipped_pairs = 0
    path_list = []
    issame_list = []
    for pair in pairs:
        if len(pair) == 3:
            path0 = os.path.join(data_dir, pair[0])
            path1 = os.path.join(data_dir, pair[1])
            if int(pair[2]) == 1:
                issame = True
            else:
                issame = False
        elif len(pair) == 4:
            path0 = os.path.join(data_dir, pair[0], pair[0] + '_' + '%04d' % int(pair[1]) + '.' + file_ext)
            path1 = os.path.join(data_dir, pair[2], pair[2] + '_' + '%04d' % int(pair[3]) + '.' + file_ext)
            issame = False
        if os.path.exists(path0) and os.path.exists(path1):  # Only add the pair if both paths exist
            path_list += (path0, path1)
            issame_list.append(issame)
        else:
            print('not exists', path0, path1)
            nrof_skipped_pairs += 1
    if nrof_skipped_pairs > 0:
        print('Skipped %d image pairs' % nrof_skipped_pairs)

    return path_list, issame_list

data_pairs = read_pairs(os.path.join(finetune_dir, 'pairs_val.txt'))
data_paths, issame_list = get_paths(lfw_dir, data_pairs, 'jpg')
print(len(data_paths))
print(len(issame_list))

lfw_bins = []
#lfw_data = nd.empty((len(lfw_paths), 3, image_size[0], image_size[1]))
i = 0
for path in data_paths:
    with open(path, 'rb') as fin:
        _bin = fin.read()
        
        lfw_bins.append(_bin)
        #img = mx.image.imdecode(_bin)
        #img = nd.transpose(img, axes=(2, 0, 1))
        #lfw_data[i][:] = img
        i+=1
        if i%1000==0:
            print('loading data', i)

with open(output, 'wb') as f:
    pickle.dump((lfw_bins, issame_list), f, protocol=pickle.HIGHEST_PROTOCOL)

41580
20790
loading data 1000
loading data 2000
loading data 3000
loading data 4000
loading data 5000
loading data 6000
loading data 7000
loading data 8000
loading data 9000
loading data 10000
loading data 11000
loading data 12000
loading data 13000
loading data 14000
loading data 15000
loading data 16000
loading data 17000
loading data 18000
loading data 19000
loading data 20000
loading data 21000
loading data 22000
loading data 23000
loading data 24000
loading data 25000
loading data 26000
loading data 27000
loading data 28000
loading data 29000
loading data 30000
loading data 31000
loading data 32000
loading data 33000
loading data 34000
loading data 35000
loading data 36000
loading data 37000
loading data 38000
loading data 39000
loading data 40000
loading data 41000


In [10]:
import scipy.io as sio
face_features = sio.loadmat('insightface/deploy/r100-softmax-African_1_20191008_no_mtcnn.mat')

In [11]:
face_features.keys()

dict_keys(['__header__', '__version__', '__globals__', 'African_5359.jpg', 'African_0226.jpg', 'African_7730.jpg', 'African_6390.jpg', 'African_7376.jpg', 'African_3326.jpg', 'African_5070.jpg', 'African_2774.jpg', 'African_3105.jpg', 'African_3873.jpg', 'African_5326.jpg', 'African_5272.jpg', 'African_4240.jpg', 'African_2586.jpg', 'African_1953.jpg', 'African_3577.jpg', 'African_7394.jpg', 'African_4970.jpg', 'African_2903.jpg', 'African_6030.jpg', 'African_6685.jpg', 'African_1663.jpg', 'African_0849.jpg', 'African_4642.jpg', 'African_6968.jpg', 'African_2859.jpg', 'African_0301.jpg', 'African_6939.jpg', 'African_3784.jpg', 'African_4698.jpg', 'African_1233.jpg', 'African_2575.jpg', 'African_4435.jpg', 'African_4314.jpg', 'African_2669.jpg', 'African_0110.jpg', 'African_1008.jpg', 'African_5477.jpg', 'African_4770.jpg', 'African_7006.jpg', 'African_1642.jpg', 'African_7182.jpg', 'African_7619.jpg', 'African_1126.jpg', 'African_0865.jpg', 'African_3757.jpg', 'African_5419.jpg', 'Afri

In [13]:
len(face_features['African_2252.jpg'][0])

512

In [16]:
import numpy as np
def cosin_metric(x1, x2):
    return np.dot(x1, x2) / (np.linalg.norm(x1) * np.linalg.norm(x2))

In [17]:
cosin_metric(face_features['African_6432.jpg'][0], face_features['African_2745.jpg'][0])

0.087745756